In [1]:
using Flows
using ToySystems
using ToySystems.NineModeSystemEq
using Optim
using PyPlot; pygui(true)

┌ Info: Recompiling stale cache file C:\Users\User\.julia\compiled\v1.2\ToySystems\chsON.ji for ToySystems [53c88820-6eff-5a8c-9a74-badd815e7be1]
└ @ Base loading.jl:1240
┌ Warning: Package ToySystems does not have Flows in its dependencies:
│ - If you have ToySystems checked out for development and have
│   added Flows as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ToySystems
└ Loading Flows into ToySystems from project dependency, future warnings for ToySystems are suppressed.


true

In [4]:
u0 = rand(9)
u0_local = copy(u0)
f  = NineModeSystem(300)

# objfun(u::AbstractVector) = (u[1] - 1)^2 + sum(u[i]^2 for i = 2:9)

# function objfun(t, u, dudt, I, dIdt)
#     return dIdt[1] = (u[1] - 1)^2 + sum(u[i]^2 for i = 2:9)
# end

# function objfun(xq::Coupled)
#     return objfun(xq[1])
# end

ϕ = flow(f, RK4(zeros(9)), TimeStepConstant(0.5))
I0 = Float64[0.0]
mon = Monitor(zeros(9), copy)
ϕ(u0_local, (0, 5000), reset!(mon))
#I0/10000
# I1 = Float64[0]
# @time ϕ(couple(u0_local, I1), (0, 10000), reset!(mon))

# times(mon)
a1 = [samples(mon)[i][1] for i = 1:length(samples(mon))]
a2 = [samples(mon)[i][2] for i = 1:length(samples(mon))]
a3 = [samples(mon)[i][3] for i = 1:length(samples(mon))]
a4 = [samples(mon)[i][4] for i = 1:length(samples(mon))]
a5 = [samples(mon)[i][5] for i = 1:length(samples(mon))]
a6 = [samples(mon)[i][6] for i = 1:length(samples(mon))]
a7 = [samples(mon)[i][7] for i = 1:length(samples(mon))]
a8 = [samples(mon)[i][8] for i = 1:length(samples(mon))]
a9 = [samples(mon)[i][9] for i = 1:length(samples(mon))];
compile = [a1,a2,a3,a4,a5,a6,a7,a8,a9]

plot(times(mon), a1)

1-element Array{PyCall.PyObject,1}:
 PyObject <matplotlib.lines.Line2D object at 0x000000004838B4C8>

In [6]:
for i in 1:9
    y = compile[i]
    fig, ax = plt.subplots()
    ax.plot(times(mon), y, Label = "a$i")
    ax.legend()
end

In [3]:
# # figure(1)
# # clf()

# # a1 = times(mon)
# # a2 = samples(mon)
# # a3 = filter(x-> (x < 0.3), a2)
# # # searchsortedfirst(a2,a3[1])
# # searchsortedfirst([1, 2, 4, 5, 5, 7], 9)
# # length(a1)
# aw = filter(x-> (x<2),[1,2,3,4,5,6,7])
# aw
# # samples(mon)[1]
# # plot(times(mon), samples(mon))
# # xlabel(L"t")
# # ylabel(L"a_1(t)")
u0_local = copy(u0)
I0 = Float64[0.0]
ϕ(couple(u0_local, I0), (10000,20000), reset!(mon))
#I0/10000
# I1 = Float64[0]
# @time ϕ(couple(u0_local, I1), (0, 10000), reset!(mon))

times(mon)
# samples(mon)

10001-element Array{Float64,1}:
 10000.0
 10001.0
 10002.0
 10003.0
 10004.0
 10005.0
 10006.0
 10007.0
 10008.0
 10009.0
 10010.0
 10011.0
 10012.0
     ⋮  
 19989.0
 19990.0
 19991.0
 19992.0
 19993.0
 19994.0
 19995.0
 19996.0
 19997.0
 19998.0
 19999.0
 20000.0

In [2]:
using PyCall
np = pyimport("numpy")
sc = pyimport("scipy")
plt = pyimport("matplotlib.pyplot");
u    = rand(9)

9-element Array{Float64,1}:
 0.3738989317383088 
 0.7777633891047808 
 0.7401733496436964 
 0.22864147550027347
 0.8073191327296285 
 0.5649767373630654 
 0.6431363252437872 
 0.5025719053887061 
 0.5465965637058177 

In [4]:
ϕ = flow(couple(f, objfun), RK4(couple(zeros(9), zeros(1))), TimeStepConstant(1))
I0 = Float64[0.0]
mon = Monitor(couple(zeros(9), zeros(1)), objfun)
ϕ(couple(u0_local, I0), (500, 10000), reset!(mon))
# #I0/10000

# I1 = Float64[0]
# @time ϕ(couple(u0_local, I1), (0, 10000), reset!(mon))

# times(mon)
samples(mon)
round(2.5)

2.0

In [3]:
include("9msctrl.jl")

ts = 0:1:500
ηs = sin.(0:2:500)
ηs2 = zeros(length(0:2:500))
dudt = zeros(9)

function firstel(u::AbstractVector)
    return u[1]
end

function firstel(t, u, dudt, I, dIdt)
    return dIdt[1] = firstel(u)
end

function firstel(xq::Coupled)
    return firstel(xq[1])
end


f = NineModeSystemCTRL(100, ts, ηs2)
#f(0.4, u, dudt)
F = flow(couple(f, firstel), RK4(couple(zeros(9), zeros(1))), TimeStepConstant(1))
I0 = Float64[0.0]
# F(couple(u,I0), (0, 100))

mon = Monitor(couple(zeros(9), zeros(1)), firstel)
times(mon)
samples(mon)
u0_local = copy(u)
# I0 = Float64[0]
F(couple(u0_local, I0), (0, 100), reset!(mon))

# figure(1)
# clf()

# plot(times(mon), samples(mon))
# xlabel(L"t")
# ylabel(L"a_1(t)")

2-element Coupled{2,Tuple{Array{Float64,1},Array{Float64,1}}}:
 [0.6345832012095037, -0.04907913624573025, 0.002775286799892493, 0.052664809724531705, -0.023948637926932595, 0.14278837054320764, 0.05047781947208772, 0.007203962386557739, -0.0050518058896502896]
 [28.7053380048022]                                                                                                                                                                                   

In [22]:
# f  = NineModeSystem(1000)
# objfun(u::AbstractVector) = (u[1] - 1)^2 + sum(u[i]^2 for i = 2:9)

# function objfun(t, u, dudt, I, dIdt)
#     return dIdt[1] = objfun(u)
# end

# function objfun(xq::Coupled)
#     return objfun(xq[1])
# end

ax.plot(times(mon), samples(mon), label = "Free")
ax.legend()

PyObject <matplotlib.legend.Legend object at 0x000000003060BE48>

In [23]:
# ϕ = flow(couple(f, objfun), RK4(couple(zeros(9), zeros(1))), TimeStepConstant(1));
xlabel(L"t")
ylabel(L"a_1(t)")

PyObject Text(31.097222222222214, 0.5, '$a_1(t)$')

In [102]:
# I0 = Float64[0.0]

1-element Array{Float64,1}:
 0.0

In [103]:
# ϕ(couple(u0, I0), (0, 10000))

2-element Coupled{2,Tuple{Array{Float64,1},Array{Float64,1}}}:
 [0.1610779955023046, 0.03322620843186161, -0.09322022365335259, -0.09954904249354425, -0.130850939521033, 0.01577832366218449, -0.03536056042512546, -0.04701700007057977, -0.11417154640561966]
 [7933.0637234252445]                                                                                                                                                                            

In [10]:
# I0/10000
m = sin(2)-sin(1)
yv = m*0.5+sin(1)

0.8753842058167891

In [11]:
ts = 0:1:500
ηs = sin.(ts)
ts1 = 0.0:0.01:15
ηs1 = sin.(ts1)
fig,ax = plt.subplots()
ax.plot(ts,ηs)
ax,plot(ts1,ηs1)
ax.plot([1,1],[0,sin(1)], linestyle = "dotted", linewidth = 1.0, color = "black")
ax.plot([2,2],[0,sin(2)], linestyle = "dotted", linewidth = 1.0, color = "black")
ax.plot([1.5,1.5],[0,yv], linestyle = "dotted", linewidth = 1.0, color = "black")
ax.scatter(1.5,yv)
xlabel(L"t")
ylabel(L"η(t)")

PyObject Text(22.347222222222214, 0.5, '$η(t)$')

In [107]:
# mon = Monitor(couple(zeros(9), zeros(1)), objfun)

Monitor{Float64,Float64,RAMStorage{Float64,Float64,3,Array{Float64,1},Array{Float64,1}},typeof(objfun)}(RAMStorage{Float64,Float64,3,Array{Float64,1},Array{Float64,1}}(Float64[], Float64[], true, 0.0), objfun, 1, (-Inf, Inf), 0, false)

In [108]:
# I0 = Float64[0]
# @time ϕ(couple(u0, I0), (0, 10000), reset!(mon))

  0.139146 seconds (209.98 k allocations: 7.955 MiB)


2-element Coupled{2,Tuple{Array{Float64,1},Array{Float64,1}}}:
 [0.13523392628156927, -0.0401165593842505, 0.019700436517505943, -0.013035132733453622, -0.028978014222952934, -0.08123509331378291, -0.015102578691530928, -0.025394043619938085, -0.0857070217754119]
 [7466.978881567882]                                                                                                                                                                                    

In [109]:
# times(mon)

10001-element Array{Float64,1}:
     0.0
     1.0
     2.0
     3.0
     4.0
     5.0
     6.0
     7.0
     8.0
     9.0
    10.0
    11.0
    12.0
     ⋮  
  9989.0
  9990.0
  9991.0
  9992.0
  9993.0
  9994.0
  9995.0
  9996.0
  9997.0
  9998.0
  9999.0
 10000.0

In [110]:
# samples(mon)

10001-element Array{Float64,1}:
 0.7573611659255434
 0.7582694786867785
 0.7598376490347148
 0.7621249986119982
 0.7651551843509019
 0.7689161294470017
 0.7733617660458608
 0.7784152697637036
 0.7839734032033591
 0.7899115743014833
 0.7960892482870168
 0.8023554169100686
 0.8085539105079947
 ⋮                 
 0.7513219569631612
 0.7521736377113729
 0.7531402131365093
 0.754213494683785 
 0.755385245292255 
 0.7566471886317493
 0.7579910146340451
 0.7594083816982174
 0.7608909160269247
 0.7624302086233933
 0.764017810546833 
 0.7656452270857004

In [111]:
# figure(1)
# clf()

# plot(times(mon), samples(mon))
# xlabel(L"t")
# ylabel(L"a_1(t)")

PyObject Text(42.722222222222214, 0.5, '$a_1(t)$')

  0.081859 seconds (95.97 k allocations: 4.379 MiB)


PyObject Text(42.597222222222214, 0.5, '$a_1(t)$')

  0.134100 seconds (95.97 k allocations: 4.379 MiB, 4.25% gc time)


PyObject Text(42.597222222222214, 0.5, '$a_1(t)$')

In [11]:
exp(2)

7.38905609893065

In [12]:
exp(1)

2.718281828459045

In [13]:
t = [1;2;3]

3-element Array{Int64,1}:
 1
 2
 3

In [14]:
exp(2)*t

3-element Array{Float64,1}:
  7.38905609893065
 14.7781121978613 
 22.16716829679195

In [1]:
ll = [1,2,3]

3-element Array{Int64,1}:
 1
 2
 3

In [2]:
ll2 = ll.^2

3-element Array{Int64,1}:
 1
 4
 9